<a href="https://colab.research.google.com/github/davidr27/456Project/blob/main/CECS456Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing libraries 
from tensorflow import keras
from keras.preprocessing import image
from numpy import asarray
import numpy as np 
import tensorflow as tf
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

In [ ]:
#download folders from kaggle
! mkdir .kaggle
! mv .kaggle /root/.kaggle
file = open('/root/.kaggle/kaggle.json', 'w')
# get key from your kaggle account
file.write('{"username":"","key":""}')
file.close()
! kaggle datasets download -d alessiocorrado99/animals10

In [ ]:
#unzip folder
!unzip animals10.zip

In [ ]:
dataset_path = 'raw-img/'

img_height = 227
img_width = 227
#batch size 32 recommended 
batch_size = 32
#obtains the training set a validation split of 0.2 is set
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    subset="training",
    labels="inferred",
    label_mode="int",
    class_names=['cane', 'cavallo', 'elefante', 'farfalla', 'gallina', 'gatto', 'mucca', 'pecora', 'ragno', 'scoiattolo'],
    color_mode="rgb",
    batch_size=32,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

#obtains the validation set a validation split of 0.2 is set
val_ds = tf.keras.utils.image_dataset_from_directory(
  dataset_path,
  subset="validation",
  labels="inferred",
  label_mode="int",
  class_names=['cane', 'cavallo', 'elefante', 'farfalla', 'gallina', 'gatto', 'mucca', 'pecora', 'ragno', 'scoiattolo'],
  batch_size=batch_size,
  image_size=(img_height, img_width),
  shuffle=True,
  seed=123,
  validation_split=0.2,
  interpolation="bilinear",
  follow_links=False,
  crop_to_aspect_ratio=False,
)

In [ ]:
#standarize the data values in the model by applying rescaling 

#Implementing AlexNet 
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D,BatchNormalization, Rescaling
#Create a sequential model 
model = Sequential()
model.add(Rescaling(1./255, input_shape=(227, 227, 3)))
model.add(Conv2D(filters=96, input_shape=(227, 227, 3),kernel_size=(11, 11),strides=(4,4),padding='valid'))
model.add(Activation('relu'))

# Pooling 
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(17))
model.add(Activation('softmax'))

model.summary()

In [ ]:
#compile the model 
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
#modify number of epochs accordingly. AlexNet showed no improvement after 20 epochs but was tested with 50. 
epochs = 50
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  verbose='auto'
)